In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torchvision import datasets, transforms
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
import sys
import copy
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
import pandas as pd
import pickle

torch.manual_seed(42)

# 하이퍼파라미터 설정
batch_size = 64
epochs = 10
lr = 0.01
PCA_dim = 8
CLS_num = 2



with open('./data/data.pkl','rb') as file:
    data = pickle.load(file)
X = data['X']
y = data['Y']



x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



def Fit_to_quantum(X,PCA_dim):
    pca = PCA(n_components=PCA_dim)
    X_pca = pca.fit_transform(X)
    return X_pca




# PyTorch Tensor로 변환
x_train_pca, y_train = torch.tensor(x_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long)
x_test_pca, y_test = torch.tensor(x_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long)


class Feature_data_loader(Dataset):
    def __init__(self,x_train,y_train):
        self.feature1 = x_train
        temp = copy.deepcopy(x_train)
        shuffle = torch.randperm(len(temp))
        self.feature2 = temp[shuffle]
        self.y1 = y_train
        temp_y = copy.deepcopy(y_train)
        self.y2 = temp_y[shuffle]

    def __len__(self):
        return len(self.feature1)
    def __getitem__(self,idx):
        input1 = self.feature1[idx]
        input2 = self.feature2[idx]
        if self.y1[idx] == self.y2[idx]:
            label = torch.tensor(1.).float()
        else:
            label = torch.tensor(0.).float()
        return [input1,input2],label


# DataLoader 생성


feature_loader = DataLoader(Feature_data_loader(x_train_pca, y_train.float()),batch_size=batch_size,shuffle=True)
test_feature_loader = DataLoader(Feature_data_loader(x_test_pca, y_test.float()),batch_size=batch_size,shuffle=False)
train_loader = DataLoader(TensorDataset(x_train_pca, y_train.float()), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(x_test_pca, y_test.float()), batch_size=batch_size, shuffle=False)


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pennylane as qml
from pennylane import numpy as np
from collections import OrderedDict
import math
from functions.training import Early_stop_train


result_list_classical = []
# Pennylane 장치 설정
dev = qml.device("default.qubit", wires=PCA_dim)


def ZZFeatureMapLayer(features, wires):
    """사용자 정의 ZZFeatureMap 레이어"""
    index = 0
    for i in wires:
        qml.Hadamard(wires=i)
        qml.RZ(features[:,index], wires=i)
        index += 1

    for j in range(0, len(wires)-1):
        qml.CNOT(wires=[j, j+1])
        qml.RZ((features[:,index]), wires=j+1)
        qml.CNOT(wires=[j, j+1])
        index+=1

def ZZFeatureMapLayer_fixed(features, wires):
    """사용자 정의 ZZFeatureMap 레이어"""
    index = 0
    for i in wires:
        qml.Hadamard(wires=i)
        qml.RZ(features[:,index], wires=i)
        index += 1
    index=0
    for j in range(0, len(wires)-1):
        qml.CNOT(wires=[j, j+1])
        qml.RZ((features[:,index])*(features[:,index+1]), wires=j+1)
        qml.CNOT(wires=[j, j+1])
        index+=1

def ansatz(params):
    for j in range(len(params)):
        # 각 큐비트에 대해 RX, RY, RZ 회전 적용
        for i in range(len(params[0])):
            qml.RY(params[j, i, 0], wires=i)
            qml.RZ(params[j, i, 1], wires=i)

        # 인접한 큐비트 간 CNOT 게이트로 엔탱글링
        if j == len(params)-1:
            pass
        else:
            for i in range(len(params[0])-1):
                qml.CNOT(wires=[i, i+1])


# 양자 레이어 정의
@qml.qnode(dev, interface='torch', diff_method="backprop")
def QuantumLayer(features,params):
    ZZFeatureMapLayer(features, wires=range(PCA_dim))
    ansatz(params)
    return qml.probs(wires=range(math.ceil(math.log2(CLS_num))))


## 양자 커널
@qml.qnode(dev, interface='torch', diff_method="backprop")
def Kernal(features1,features2):
    ZZFeatureMapLayer(features1, wires=range(PCA_dim))
    qml.adjoint(ZZFeatureMapLayer)(features2,wires=range(PCA_dim))
    return qml.probs(wires=range(PCA_dim))

@qml.qnode(dev, interface='torch', diff_method="backprop")
def Kernal_fix(features1,features2):
    ZZFeatureMapLayer_fixed(features1, wires=range(PCA_dim))
    qml.adjoint(ZZFeatureMapLayer_fixed)(features2,wires=range(PCA_dim))
    return qml.probs(wires=range(PCA_dim))


class Feature_model(nn.Module):
    def __init__(self):
        super(Feature_model,self).__init__()
        self.cls = nn.Sequential(OrderedDict([('cls1', nn.Linear(PCA_dim,PCA_dim*8)),
                                              ('relu1', nn.ReLU()),('cls2', nn.Linear(PCA_dim*8,PCA_dim*8)),
                                              ('relu2', nn.ReLU()),('cls3', nn.Linear(PCA_dim*8,PCA_dim*8)),
                                              ('relu3', nn.ReLU()),('cls4', nn.Linear(PCA_dim*8,PCA_dim*8)),
                                              ('relu4', nn.ReLU()),('cls5', nn.Linear(PCA_dim*8,PCA_dim*2-1))
                                              ]))
        self.Kernal = Kernal
    def forward(self,inputs):
        epsilon = 1e-6
        input1 = inputs[0]
        input2 = inputs[1]
        input1 = self.cls(input1)*np.pi
        input2 = self.cls(input2)*np.pi
        output = self.Kernal(input1,input2)
        output = output.type(torch.float32)
        return output[:,0].clamp(min=epsilon, max=1-epsilon)





# 하이브리드 모델 정의
class HybridModel(nn.Module):
    def __init__(self):
        super(HybridModel, self).__init__()
        self.cls = feature_model.cls

        self.quantum_layer = QuantumLayer
        self.Q_params = nn.Parameter((torch.rand([PCA_dim,PCA_dim,2])*2-1)*np.pi,requires_grad=True)
    def forward(self, x):
        epsilon = 1e-6
        x = self.cls(x)*np.pi
        #print(qml.draw(self.quantum_layer)(x,self.Q_params))
        quantum_output = self.quantum_layer(x,self.Q_params)
        quantum_output = quantum_output.type(torch.float32)
        return quantum_output[:,0].clamp(min=epsilon, max=1-epsilon)

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.cls_layer_1 = nn.Linear(PCA_dim,PCA_dim*PCA_dim)
        self.cls_layer_2 = nn.Linear(PCA_dim*PCA_dim,PCA_dim*PCA_dim-1)
        self.output_layer = nn.Linear(PCA_dim*PCA_dim-1,PCA_dim)
    def forward(self, x):
        x = self.cls_layer_1(x)
        x = nn.ReLU()(x)
        x = self.cls_layer_2(x)
        x = nn.ReLU()(x)
        output = self.output_layer(x)
        return output
# 모델, 손실 함수, 최적화 설정


feature_model = Feature_model(); criterion = nn.BCELoss()
#model = Model(); criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(feature_model.parameters(), lr=0.001)


# 모델 학습 및 평가
train_process = Early_stop_train(feature_model, optimizer, criterion)
train_process.train_model(feature_loader,test_feature_loader,epochs=50,res=15)
 
model = HybridModel(); criterion = nn.BCELoss()
for param in model.cls.parameters():
    param.requires_grad = False
#model.load_state_dict(para_dict)
optimizer = optim.Adam(model.parameters(), lr=0.01)
print('\n\n Test start \n\n')
train_process = Early_stop_train(model, optimizer, criterion)
train_process.train_model(train_loader,test_loader,epochs=50,res=5)

_,acc = train_process.test(test_loader)
result_list_classical.append(acc)
print(f"Test Accuracy: {acc:.2f}")

model = HybridModel(); criterion = nn.BCELoss()
optimizer = optim.Adam([model.Q_params], lr=0.01)
print('\n\n Test start \n\n')
train_process = Early_stop_train(model, optimizer, criterion)
train_process.train_model(train_loader,test_loader,epochs=50,res=5)

_,acc = train_process.test(test_loader)
result_list_classical.append(acc)
print(f"Test Accuracy: {acc:.2f}")


In [ ]:
## 커널 메소드
from functions.training import Kernal_method

kernal = Kernal_method(feature_model)

kernel_matrix,test_kernel_matrix,alpha,labels = kernal.train(x_train_pca,y_train,x_test_pca,y_test)